In [125]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.examples.ubc_decorators import *
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python.examples.straightforward_agent_2 import StraightforwardAgent as StraightforwardAgent2
from open_spiel.python import rl_environment


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [126]:
game_name = 'large_game_2_one_profile'
game_config = json.load(open(f"{os.environ['CLOCK_AUCTION_CONFIG_DIR']}/{game_name}.json"))
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename=f'{game_name}.json'))


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2_one_profile.json
Done config parsing


In [127]:
# x = np.array(game.new_initial_state().child(0).child(0).child(10).child(10).child(10).child(10).information_state_tensor())
# x[x.nonzero()]

In [140]:
fix_seeds(1234)
env = rl_environment.Environment(game)
num_actions = env.action_spec()["num_actions"]
time_step = env.reset()

agents = [TakeSingleActionDecorator(StraightforwardAgent2(i, game_config, num_actions), num_actions) for i in range(game.num_players())]

rewards = defaultdict(list)
allocations = defaultdict(list)
payments = defaultdict(list)
episode_length = 0
episode_rewards = defaultdict(int)
num_players, num_actions, num_products = game_spec(game, game_config)

while not time_step.last():
    episode_length += 1
            
    player_id = time_step.observations["current_player"]
    # print(time_step.observations.keys())
    agent = agents[player_id]
    agent_output = agent.step(time_step)
    print(player_id, agent_output.action)
    action_list = [agent_output.action]
    time_step = env.step(action_list)

for i, agent in enumerate(agents):
    agent.step(time_step, is_evaluation=True)
    episode_rewards[i] += time_step.rewards[i] 
    rewards[i] = episode_rewards[i]
    infostate = time_step.observations['info_state'][i]
    payment, allocation = payment_and_allocation(num_players, num_actions, num_products, infostate)
    payments[i] = payment
    allocations[i] = allocation
    
print('Outcome')
print(rewards)
print(allocations)
print(payments)
print(episode_length)

Writing LP
only 1 feasible bundle
0 0
Writing LP
only 1 feasible bundle
1 0
Outcome
defaultdict(<class 'list'>, {0: 0.0, 1: 0.0})
defaultdict(<class 'list'>, {0: [0.0, 0.0, 0.0], 1: [0.0, 0.0, 0.0]})
defaultdict(<class 'list'>, {0: 0.0, 1: 0.0})
2
